In [16]:
import os
import tempfile
from pathlib import Path
from copy import copy
import win32com.client as win32
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
import pandas as pd
import numpy as np
# ==== 设置路径 ====
inp = Path(r'.\data\原始数据模板_财务分析_2025Q1.xlsx').resolve()
out = Path(r'.\data\原始数据模板_财务分析_2025Q1_v1.xlsx').resolve()

# ==== 指标列表 ====
new_metrics = [
    '流动比率（扣除预收账款）', '现金比率（扣除预收账款）', '资产负债率', 'Net Gearing',
    '应收/总资产', '货币资金/有息负债',
    '同比',
    '应收款', '应收账款', '预收账款', '现金', '存货', '应付票据和账款（短期）',
    '环比',
    '应收账款（环比）', '其他应收款', '固定资产', '资产总计', '应付票据和账款（短期）（环比）', '存货（环比）', '现金(环比）', '应收变化/收入变化',
    '杜邦分析',
    'ROE(pro forma)', '净利率', '总资产周转率', '杠杆', '存货周转率', '应收账款周转率'
]

new_profit_metrics = [
    '扣非归母净利润',
    # 常规比率
    '已获利息倍数', '毛利率', '销售费用/收入', '管理费用/收入', '研发费用/收入',
    '财务费用/收入', '费用/收入', '三费/毛利润', '营业利润率', '净利率',
    
    # 分组标题
    '同比',
    
    # 同比指标
    '营业收入', '营业总成本', '销售费用', '管理费用', '研发费用',
    '财务费用', '资产减值损失', '公允价值变动收益', '投资收益', '其中:对联营企业和合营企业的投资收益',
    
    # 单项
    '营业利润', '归母净利润', '归母扣非净利润 （估算，不含投资收益）',
    '归母扣非净利润 %（估算，不含投资收益）同比',
    
    # 分组标题
    '环比',
    
    # 环比指标，注意加“（环比）”后缀
    '营业收入（环比）', '归母净利润（环比）', '归母扣非净利润 %（估算，不含投资收益）（环比）',
    '应收应付率',
    '核心利润',
    '每季度收入（同比）',
    '每季度利润（同比）',
    '每季度核心利润（同比）',
    '每季度归母扣非净利润（同比）（估算，不含投资收益）',
    '每季度核心利润（环比）'
]
cashflow_metrics = [
    # 常规
    '经营活动现金净流量/折旧摊销',
    '经营现金流净额/净利润',
    '销售商品、提供劳务收到的现金/营业收入',
    '购建固定资产、无形资产和其他长期资产所支付的现金/经营现金流净额',
    '购建固定资产、无形资产和其他长期资产所支付的现金/收入',
    '现金余额/(投资支出+现金分红)',
    '销售商品、提供劳务收到的现金+应收账款，应收票据增加额/营业收入',

    # 同比
    '销售商品、提供劳务收到的现金（同比）',
    '经营活动产生的现金流量净额（同比）',

    # 环比
    '销售商品、提供劳务收到的现金（环比）',
    '经营活动产生的现金流量净额（环比）',

    # 每季度（前空两行）
    '每季度销售商品、提供劳务收到的现金（同比)',
    '每季度经营活动产生的现金流量净额（同比）'
]

percentage_metrics = {
    '资产负债率',
    'Net Gearing',
    '应收/总资产',
    '应收款',
    '应收账款',
    '预收账款',
    '现金',
    '存货',
    '应付票据和账款（短期）',
    '应收账款（环比）', '其他应收款', '固定资产', '资产总计', '应付票据和账款（短期）（环比）', '存货（环比）', 
    '现金(环比）', '应收变化/收入变化', 'ROE(pro forma)', '净利率', '总资产周转率', '杠杆',
    '毛利率', '销售费用/收入', '管理费用/收入', '研发费用/收入',
    '财务费用/收入', '费用/收入', '三费/毛利润', '营业利润率', '净利率',
    '营业收入', '营业总成本', '销售费用', '管理费用', '研发费用',
    '财务费用', '资产减值损失', '公允价值变动收益', '投资收益', '其中:对联营企业和合营企业的投资收益',
        '营业利润', '归母净利润', 
    '归母扣非净利润 %（估算，不含投资收益）同比',
    '营业收入（环比）', '归母净利润（环比）', '归母扣非净利润 %（估算，不含投资收益）（环比）',
    '应收应付率',
    '每季度收入（同比）',
    '每季度利润（同比）',
    '每季度核心利润（同比）',
    '每季度归母扣非净利润（同比）（估算，不含投资收益）',
    '每季度核心利润（环比）'
}

# 计算同比
def yoy_series(row: pd.Series, offset=4):
    result = []
    for i in range(len(row)):
        if i + offset < len(row):
            denom = row[i + offset]
            if denom == 0 or pd.isna(denom):
                result.append(np.nan)
            else:
                result.append(row[i] / denom - 1)
        else:
            result.append(np.nan)
    return pd.Series(result)

#计算环比
def qoq_series(row: pd.Series, offset=1):
    result = []
    for i in range(len(row)):
        if i + offset < len(row):
            denom = row[i + offset]
            if denom == 0 or pd.isna(denom):
                result.append(np.nan)
            else:
                result.append(row[i] / denom - 1)
        else:
            result.append(np.nan)
    return pd.Series(result)

# ==== 修改后的 add_new_metrics 函数
def add_new_metrics(ws_formula, new_metrics):
    max_row = ws_formula.max_row
    max_col = ws_formula.max_column

    row_white = ws_formula[max_row - 1]
    row_gray = ws_formula[max_row]
    assert len(row_white) == len(row_gray)

    insert_blank_after = {
        '扣非归母净利润',
        '货币资金/有息负债',
        '应付票据和账款（短期）',
        '应收变化/收入变化',
        '杠杆',
        '净利率',
        '归母扣非净利润 %（估算，不含投资收益）同比',
        '销售商品、提供劳务收到的现金+应收账款，应收票据增加额/营业收入',
        '经营活动产生的现金流量净额（同比）',
        '经营活动产生的现金流量净额（环比）',
    }

    insert_two_blank_before = {
        '每季度销售商品、提供劳务收到的现金（同比)',
        '每季度经营活动产生的现金流量净额（同比）'
    }

    green_fill_metrics = {
        '现金', '存货', '现金(环比）', '应收变化/收入变化',
        '每季度销售商品、提供劳务收到的现金（同比)',
        '每季度经营活动产生的现金流量净额（同比）'
    }

    yellow_fill_metrics = {'存货周转率', '应收账款周转率'}
    group_titles = {'同比', '环比'}

    green_fill = PatternFill(fill_type="solid", start_color="C6EFCE", end_color="C6EFCE")
    yellow_fill = PatternFill(fill_type="solid", start_color="FFF2CC", end_color="FFF2CC")

    current_row = max_row + 1
    for _ in range(2):
        for col in range(1, max_col + 1):
            ref_cell = row_white[col - 1]
            cell = ws_formula.cell(row=current_row, column=col)
            cell.value = '' if col == 1 else None
            cell.font = copy(ref_cell.font)
            cell.alignment = copy(ref_cell.alignment)
            cell.border = copy(ref_cell.border)
            cell.fill = copy(ref_cell.fill)
            cell.number_format = copy(ref_cell.number_format)
        current_row += 1

    row_map = {}
    use_gray = False

    for metric in new_metrics:
        template_row = row_gray if use_gray else row_white
        use_gray = not use_gray

        if metric in group_titles:
            for col in range(1, max_col + 1):
                ref_cell = template_row[col - 1]
                cell = ws_formula.cell(row=current_row, column=col)
                cell.value = metric if col == 1 else None
                cell.font = copy(ref_cell.font)
                cell.alignment = copy(ref_cell.alignment)
                cell.border = copy(ref_cell.border)
                cell.fill = copy(ref_cell.fill)
                cell.number_format = copy(ref_cell.number_format)
            current_row += 1
            continue

        if metric in insert_two_blank_before:
            for _ in range(2):
                for col in range(1, max_col + 1):
                    ref_cell = row_white[col - 1]
                    cell = ws_formula.cell(row=current_row, column=col)
                    cell.value = '' if col == 1 else None
                    cell.font = copy(ref_cell.font)
                    cell.alignment = copy(ref_cell.alignment)
                    cell.border = copy(ref_cell.border)
                    cell.fill = copy(ref_cell.fill)
                    cell.number_format = copy(ref_cell.number_format)
                current_row += 1

        for col in range(1, max_col + 1):
            ref_cell = template_row[col - 1]
            cell = ws_formula.cell(row=current_row, column=col)
            cell.value = metric if col == 1 else None
            cell.font = copy(ref_cell.font)
            cell.alignment = copy(ref_cell.alignment)
            cell.border = copy(ref_cell.border)
            cell.fill = copy(ref_cell.fill)
            cell.number_format = copy(ref_cell.number_format)
            if col == 1:
                if metric in green_fill_metrics:
                    cell.fill = green_fill
                elif metric in yellow_fill_metrics:
                    cell.fill = yellow_fill

        row_map[metric] = current_row
        current_row += 1

        if metric in insert_blank_after:
            for col in range(1, max_col + 1):
                ref_cell = row_white[col - 1]
                cell = ws_formula.cell(row=current_row, column=col)
                cell.value = '' if col == 1 else None
                cell.font = copy(ref_cell.font)
                cell.alignment = copy(ref_cell.alignment)
                cell.border = copy(ref_cell.border)
                cell.fill = copy(ref_cell.fill)
                cell.number_format = copy(ref_cell.number_format)
            current_row += 1

    return row_map



# 计算 应收变化/收入变化
def calc_receivable_change_ratio_multicol(df_main, df_profit):
    result = []
    for col in range(df_main.shape[1] - 1):  # 要防止 col+1 越界
        # === 分子：当前列应收 - 下一列应收 ===
        curr_sum = df_main.iloc[13:16, col].sum()
        next_sum = df_main.iloc[13:16, col + 1].sum()
        delta = curr_sum - next_sum  # 当前列减下一列

        # === 分母判断值
        current_flag = df_main.iloc[0, col]

        income_curr = df_profit.iloc[9, col]
        income_next = df_profit.iloc[9, col + 1]

        if current_flag == 1:
            base = income_curr
        else:
            base = income_curr - income_next  # 当前列减下一列

        if base == 0 or pd.isna(base):
            result.append(np.nan)
        else:
            result.append(delta / base)

    result.append(np.nan)  # 最后一列无法比较下一列，对应为空
    return pd.Series(result)
#ROE(pro forma)
def calc_roe(df, df_profit):
    roe_list = []
    for i in range(df.shape[1]):
        flag = df.iloc[0, i]
        net_profit = df_profit.iloc[58, i]
        equity = df.iloc[140, i]
        if equity == 0 or pd.isna(equity):
            roe_list.append(np.nan)
            continue
        if flag == 1:
            value = net_profit / 1 * 4
        elif flag == 2:
            value = net_profit / 2 * 4
        elif flag == 3:
            value = net_profit / 3 * 4
        elif flag == 4:
            value = net_profit / 4 * 4
        else:
            value = np.nan
        roe_list.append(value / equity)
    return pd.Series(roe_list)

#净利率
def calc_net_margin(df_profit):
    return df_profit.iloc[54, :] / df_profit.iloc[8, :]

#总资产周转率
def calc_asset_turnover(df, df_profit):
    result = []
    for i in range(df.shape[1]):
        flag = df.iloc[0, i]
        revenue = df_profit.iloc[8, i]  # B9
        if flag == 1:
            revenue = revenue / 1 * 4
        elif flag == 2:
            revenue = revenue / 2 * 4
        elif flag == 3:
            revenue = revenue / 3 * 4
        elif flag == 4:
            revenue = revenue / 4 * 4
        else:
            revenue = np.nan
        try:
            avg_asset = (df.iloc[71, i] + df.iloc[71, 4]) / 2  # B72 和 F72
        except:
            avg_asset = np.nan
        if avg_asset == 0 or pd.isna(avg_asset):
            result.append(np.nan)
        else:
            result.append(revenue / avg_asset)
    return pd.Series(result)
#杠杆
def calc_leverage(df):
    result = []
    for i in range(df.shape[1]):
        try:
            avg_asset = (df.iloc[71, i] + df.iloc[71, 4]) / 2
        except:
            avg_asset = np.nan
        equity = df.iloc[140, i]
        if equity == 0 or pd.isna(equity):
            result.append(np.nan)
        else:
            result.append(avg_asset / equity)
    return pd.Series(result)
#存货周转率'
def calc_inventory_turnover(df, df_profit):
    result = []
    for i in range(df.shape[1]):
        flag = df.iloc[0, i]
        cogs = df_profit.iloc[15, i]  # Excel第16行 → iloc[15]
        if flag == 1:
            cogs = cogs / 1 * 4
        elif flag == 2:
            cogs = cogs / 2 * 4
        elif flag == 3:
            cogs = cogs / 3 * 4
        elif flag == 4:
            cogs = cogs / 4 * 4
        else:
            cogs = np.nan
        try:
            avg_inventory = (df.iloc[22, i] + df.iloc[22, 4]) / 2  # B23,F23 → 行22，列i和4
        except:
            avg_inventory = np.nan
        if avg_inventory == 0 or pd.isna(avg_inventory):
            result.append(np.nan)
        else:
            result.append(cogs / avg_inventory)
    return pd.Series(result)
#'应收账款周转率
def calc_ar_turnover(df, df_profit):
    result = []
    for i in range(df.shape[1]):
        flag = df.iloc[0, i]
        revenue = df_profit.iloc[8, i]  # Excel第9行 → iloc[8]
        if flag == 1:
            revenue = revenue / 1 * 4
        elif flag == 2:
            revenue = revenue / 2 * 4
        elif flag == 3:
            revenue = revenue / 3 * 4
        elif flag == 4:
            revenue = revenue / 4 * 4
        else:
            revenue = np.nan
        try:
            avg_ar = (df.iloc[14, i] + df.iloc[14, 4]) / 2  # B15,F15 → 行14，列i和4
        except:
            avg_ar = np.nan
        if avg_ar == 0 or pd.isna(avg_ar):
            result.append(np.nan)
        else:
            result.append(revenue / avg_ar)
    return pd.Series(result)
#计算函数对于sheet2
def compute_profit_metrics(df_profit: pd.DataFrame, df_main: pd.DataFrame) -> dict:
    results = {}

    # 用 .iloc[:, i] 表示第 i 列
    n_cols = df_profit.shape[1]

    def col(i, row):
        return df_profit.iloc[row, i]

    def prev(i, row):
        return df_profit.iloc[row, i + 1] if i + 1 < n_cols else np.nan

    def prev2(i, row):
        return df_profit.iloc[row, i + 2] if i + 2 < n_cols else np.nan
    
    def prev4(i, row):
        return df_profit.iloc[row, i + 4] if i + 4 < n_cols else np.nan
     
    def prev5(i, row):
        return df_profit.iloc[row, i + 5] if i + 5 < n_cols else np.nan
       
    for i in range(n_cols):
        flag = df_profit.iloc[0, i]
        
        # 扣非归母净利润 =B55-B45+B46-B32
        kf = col(i, 54) - col(i, 44) + col(i, 45) - col(i, 31)
        results.setdefault('扣非归母净利润', []).append(kf)

        # 已获利息倍数 =(B50-B21)/B21
        ebit = col(i, 49) - col(i, 20)
        results.setdefault('已获利息倍数', []).append(ebit / col(i, 20) if col(i, 20) else np.nan)

        # 毛利率=1-B16/B9
        results.setdefault('毛利率', []).append(1 - col(i, 15) / col(i, 8) if col(i, 8) else np.nan)

        # 销售费用/收入 =B18/B9
        results.setdefault('销售费用/收入', []).append(col(i, 17) / col(i, 8) if col(i, 8) else np.nan)

        results.setdefault('管理费用/收入', []).append(col(i, 18) / col(i, 8) if col(i, 8) else np.nan)
        results.setdefault('研发费用/收入', []).append(col(i, 19) / col(i, 8) if col(i, 8) else np.nan)
        results.setdefault('财务费用/收入', []).append(col(i, 20) / col(i, 8) if col(i, 8) else np.nan)

        # 费用/收入 =
        sf_sum = sum([col(i, r) for r in range(17, 21 + 1)])  # B18~B21 是行 17~20
        income = col(i, 8)  # B9 是行 8
        results.setdefault('费用/收入', []).append(sf_sum / income if income else np.nan)


        # 三费/毛利润 =SUM(B18:B21)/(B9-B16)
        sf = sum([col(i, r) for r in range(17, 21 + 1)])
        margin = col(i, 8) - col(i, 15)
        results.setdefault('三费/毛利润', []).append(sf / margin if margin else np.nan)

        # 营业利润率 =B44/B9
        results.setdefault('营业利润率', []).append(col(i, 43) / col(i, 8) if col(i, 8) else np.nan)

        # 净利率 =B55/B9
        results.setdefault('净利率', []).append(col(i, 54) / col(i, 8) if col(i, 8) else np.nan)

        # ====== 同比类 ======
        def yoy(row):  # 当前列 vs 第 i+4 列
            return (col(i, row) / prev4(i, row) - 1) if prev4(i, row) else np.nan

        yoy_rows = {
            '营业收入': 8, '营业总成本': 14, '销售费用': 17,
            '管理费用': 18, '研发费用': 19, '财务费用': 20,
            '资产减值损失': 37, '公允价值变动收益': 36,
            '投资收益': 32, '其中:对联营企业和合营企业的投资收益': 33,
            '营业利润': 43, '归母净利润': 58
        }

        for name, row in yoy_rows.items():
            results.setdefault(name, []).append(yoy(row))

        # 归母扣非净利润 （估算，不含投资收益） = B59-B45+B46-B32-B33
        adjusted = col(i, 58) - col(i, 44) + col(i, 45) - col(i, 31) - col(i, 32)
        results.setdefault('归母扣非净利润 （估算，不含投资收益）', []).append(adjusted)

        try:
            # 当前值 = B59 - B45 + B46 - B32 - B33
            curr = col(i, 58) - col(i, 44) + col(i, 45) - col(i, 31) - col(i, 32)
            # 去年同期 = F59 - F45 + F46 - F32 - F33
            base = prev(i, 58) - prev(i, 44) + prev(i, 45) - prev(i, 31) - prev(i, 32)
            value = (curr / base - 1) if base else np.nan
        except IndexError:
            value = np.nan

        results.setdefault('归母扣非净利润 %（估算，不含投资收益）同比', []).append(value)


        # ====== 环比类 ======
        def qoq(val_curr, val_prev, val_prev2):
            if flag == 1:
                return val_curr / (val_prev - val_prev2) - 1 if val_prev - val_prev2 else np.nan
            elif flag == 2:
                return (val_curr - val_prev) / val_prev if val_prev else np.nan
            elif flag in (3, 4):
                return (val_curr - val_prev) / (val_prev - val_prev2) - 1 if (val_prev - val_prev2) else np.nan
            return np.nan

        results.setdefault('营业收入（环比）', []).append(qoq(col(i, 8), prev(i, 8), prev2(i, 8)))
        results.setdefault('归母净利润（环比）', []).append(qoq(col(i, 58), prev(i, 58), prev2(i, 58)))
        try:
            curr = col(i, 58) - col(i, 44) + col(i, 45) - col(i, 31) - col(i, 32)
            p1 = prev(i, 58) - prev(i, 44) + prev(i, 45) - prev(i, 31) - prev(i, 32)
            p2 = prev2(i, 58) - prev2(i, 44) + prev2(i, 45) - prev2(i, 31) - prev2(i, 32)

            if flag == 1:
                val = curr / (p1 - p2) - 1 if (p1 - p2) else np.nan
            elif flag == 2:
                val = (curr - p1) / p1 if p1 else np.nan
            elif flag in (3, 4):
                val = (curr - p1) / (p1 - p2) - 1 if (p1 - p2) else np.nan
            else:
                val = np.nan
        except IndexError:
            val = np.nan

        results.setdefault('归母扣非净利润 %（估算，不含投资收益）（环比）', []).append(val)

        # 应收应付率
        if i >= df_main.shape[1]:
            results.setdefault('应收应付率', []).append(np.nan)
        else:
            ar_sum = df_main.iloc[13:16, i].sum()
            ap_sum = df_main.iloc[77:80, i].sum()
            results.setdefault('应收应付率', []).append(
                (ar_sum - ap_sum) / ap_sum if ap_sum else np.nan
            )

        # 核心利润 =B9-B16-B17-B18-B19-B20-B21
        cp = col(i, 8) - sum([col(i, r) for r in range(15, 21)])
        results.setdefault('核心利润', []).append(cp)

        # 每季度收入（同比） = 特殊逻辑
        def quarter_yoy(row):
            if flag == 1:
                return (col(i, row) / prev4(i, row)) - 1 if prev4(i, row) else np.nan
            else:
                curr_delta = col(i, row) - prev(i, row)
                prev_delta = prev4(i, row) - prev5(i, row)
                return (curr_delta / prev_delta) - 1 if prev_delta else np.nan

        results.setdefault('每季度收入（同比）', []).append(quarter_yoy(8))
        results.setdefault('每季度利润（同比）', []).append(quarter_yoy(58))
        try:
            curr = col(i, 8) - sum([col(i, r) for r in range(15, 21)])
            p1 = prev(i, 8) - sum([prev(i, r) for r in range(15, 21)])
            p4 = prev4(i, 8) - sum([prev4(i, r) for r in range(15, 21)])
            p5 = prev2(i, 8) - sum([prev5(i, r) for r in range(15, 21)])

            if flag == 1:
                val = curr / p4 - 1 if p4 else np.nan
            else:
                base = p5
                if base == 0 or pd.isna(base):
                    val = np.nan
                else:
                    val = (curr - p1) / (p4 - base) - 1 if (p4 - base) else np.nan
        except IndexError:
            val = np.nan

        results.setdefault('每季度核心利润（同比）', []).append(val)

        try:
            curr = col(i, 58) - col(i, 44) + col(i, 45) - col(i, 31) - col(i, 32)
            p1 = prev(i, 58) - prev(i, 44) + prev(i, 45) - prev(i, 31) - prev(i, 32)
            p5 = prev5(i, 58) - prev5(i, 44) + prev5(i, 45) - prev5(i, 31) - prev5(i, 32)
            p4 = prev4(i, 58) - prev4(i, 44) + prev4(i, 45) - prev4(i, 31) - prev4(i, 32)
            if flag == 1:
                val = curr / p4 - 1 if p1 else np.nan
            else:
                val = (curr - p1) / (p4 - p5) - 1 \
                    if (p2 - p5) else np.nan
        except IndexError:
            val = np.nan

        results.setdefault('每季度归母扣非净利润（同比）（估算，不含投资收益）', []).append(val)


        # 每季度核心利润（环比）
        try:
            curr = col(i, 8) - sum([col(i, r) for r in range(15, 21)])
            p1 = prev(i, 8) - sum([prev(i, r) for r in range(15, 21)])
            p2 = prev2(i, 8) - sum([prev2(i, r) for r in range(15, 21)])

            if flag == 1:
                val = curr / (p1 - p2) - 1 if (p1 - p2) else np.nan
            elif flag == 2:
                val = (curr - p1) / p1 - 1 if p1 else np.nan
            else:
                val = (curr - p1) / (p1 - p2) - 1 if (p1 - p2) else np.nan

        except IndexError:
            val = np.nan

        results.setdefault('每季度核心利润（环比）', []).append(val)


    # 转换为 Series
    return {k: pd.Series(v) for k, v in results.items()}

# ==== 步骤 1：用 Excel 自动清除外链 ====
def break_external_links(input_path: str) -> str:
    excel = win32.gencache.EnsureDispatch('Excel.Application')
    excel.Visible = False
    excel.DisplayAlerts = False
    tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".xlsx")
    tmp.close()
    tmp_path = tmp.name

    wb = excel.Workbooks.Open(Filename=input_path, UpdateLinks=0, ReadOnly=True)
    links = wb.LinkSources(Type=1)
    if links:
        for link in links:
            wb.BreakLink(Name=link, Type=1)
    wb.SaveAs(tmp_path, FileFormat=51)  # 明确指定 xlsx 格式
    wb.Close()
    excel.Quit()
    return tmp_path

# ==== 步骤 2：数据处理并保存新文件 ====
import pandas as pd

def process_excel(cleaned_path: str, output_path: str):
    wb_formula = load_workbook(cleaned_path)
    wb_values = load_workbook(cleaned_path, data_only=True)
    ws_formula = wb_formula.active
    ws_values = wb_values.active

    # 加载第二个sheet
    sheet_names = pd.ExcelFile(cleaned_path).sheet_names
    second_sheet_name = sheet_names[1]  # 索引从0开始，1表示第二个sheet
    df_profit_raw = pd.read_excel(cleaned_path, sheet_name=second_sheet_name, header=None)
    df_profit = df_profit_raw.iloc[:, 1:].apply(pd.to_numeric, errors='coerce').fillna(0)

    # 加载第三个sheet：现金流
    df_cashflow_raw = pd.read_excel(cleaned_path, sheet_name=sheet_names[2], header=None)
    df_cashflow = df_cashflow_raw.iloc[:, 1:].apply(pd.to_numeric, errors='coerce').fillna(0)


    # 替换 B3 为显示值
    ws_formula['B3'].value = ws_values['B3'].value

    # 验证 A5 是否正确
    if str(ws_values['A5'].value).strip() != '资产负债表(ORIG,元)':
        raise ValueError(f"第5行第1列非预期: {ws_values['A5'].value}")

    # 插入新指标并获取行号映射
    row_map = add_new_metrics(ws_formula, new_metrics)

    # === 读取原始数据并转换为数值 ===
    df_raw = pd.read_excel(cleaned_path, header=None)
    df = df_raw.iloc[:, 1:].apply(pd.to_numeric, errors='coerce').fillna(0)
    # 插入利润表指标（第二个 sheet）
    ws_profit = wb_formula.worksheets[1]

    ws_cashflow = wb_formula.worksheets[2]
    add_new_metrics(ws_cashflow, cashflow_metrics)
    # ==== 定义计算公式  ====
    formulas = {
        '流动比率（扣除预收账款）': lambda d: d.iloc[37] / (d.iloc[105] - d.iloc[79]),
        '现金比率（扣除预收账款）': lambda d: d.iloc[9]  / (d.iloc[105] - d.iloc[79]),
        '资产负债率':             lambda d: d.iloc[123] / d.iloc[71],
        'Net Gearing':            lambda d: (
                                        d.iloc[73] + d.iloc[89] + d.iloc[107] +
                                        d.iloc[77] + d.iloc[78] + d.iloc[93] +
                                        d.iloc[92] + d.iloc[108] - d.iloc[9]
                                    ) / d.iloc[140],
        '应收/总资产':             lambda d: (
                                        d.iloc[13] + d.iloc[14] + d.iloc[15] + d.iloc[17]
                                    ) / d.iloc[71],
        '货币资金/有息负债':       lambda d: d.iloc[9] / (
                                        d.iloc[73] + d.iloc[92] + d.iloc[107] + d.iloc[108]
                                    ),
        
        '应收款': lambda d: yoy_series((d.iloc[13] + d.iloc[14] + d.iloc[15] + d.iloc[17]).reset_index(drop=True)),
        '应收账款': lambda d: yoy_series(d.iloc[14].reset_index(drop=True)),
        '预收账款': lambda d: yoy_series(d.iloc[79].reset_index(drop=True)),
        '现金':     lambda d: yoy_series(d.iloc[9].reset_index(drop=True)),
        '存货':     lambda d: yoy_series(d.iloc[22].reset_index(drop=True)),
        '应付票据和账款（短期）': lambda d: yoy_series((d.iloc[77] + d.iloc[78]).reset_index(drop=True)),

        # 环比
        '应收账款（环比）': lambda d: qoq_series(d.iloc[14].reset_index(drop=True)),
        '其他应收款':       lambda d: qoq_series(d.iloc[17].reset_index(drop=True)),
        '固定资产':         lambda d: qoq_series(d.iloc[51].reset_index(drop=True)),
        '资产总计':         lambda d: qoq_series(d.iloc[71].reset_index(drop=True)),
        '应付票据和账款（短期）（环比）': lambda d: qoq_series((d.iloc[77] + d.iloc[78]).reset_index(drop=True)),
        '存货（环比）':     lambda d: qoq_series(d.iloc[22].reset_index(drop=True)),
        '现金(环比）':      lambda d: qoq_series(d.iloc[9].reset_index(drop=True)),
        '应收变化/收入变化': lambda d, dfp: calc_receivable_change_ratio_multicol(d, dfp),
        'ROE(pro forma)': lambda d: calc_roe(d, df_profit),
        '净利率': lambda d: calc_net_margin(df_profit),
        '总资产周转率': lambda d: calc_asset_turnover(d, df_profit),
        '杠杆': lambda d: calc_leverage(d),
        '存货周转率': lambda d: calc_inventory_turnover(d, df_profit),
        '应收账款周转率': lambda d: calc_ar_turnover(d, df_profit),

    }

    # === 计算公式 ===
    computed_results = {}
    for metric, func in formulas.items():
        try:
            if metric == '应收变化/收入变化':
                result = func(df, df_profit)
            else:
                result = func(df)
            computed_results[metric] = result
        except Exception as e:
            print(f"计算 {metric} 出错: {e}")
            computed_results[metric] = None

    # === 写入结果 ===

    for metric, row in row_map.items():
        if metric in computed_results:
            values = computed_results[metric]
            if isinstance(values, pd.Series):
                for i, v in enumerate(values):
                    cell = ws_formula.cell(row=row, column=2 + i)
                    if pd.notna(v):
                        if metric in percentage_metrics:
                            cell.value = float(v)
                            cell.number_format = '0.0%'
                        else:
                            cell.value = float(v)
                            cell.number_format = '0.00'
                    else:
                        cell.value = None
            else:
                cell = ws_formula.cell(row=row, column=2)
                if pd.notna(values):
                    if metric in percentage_metrics:
                        cell.value = float(values)
                        cell.number_format = '0.0%'
                    else:
                        cell.value = float(values)
                        cell.number_format = '0.00'
                else:
                    cell.value = None

    # ==== 计算利润表指标 ====
    profit_results = compute_profit_metrics(df_profit, df)

    # 写入 ws_profit
    profit_row_map = add_new_metrics(ws_profit, new_profit_metrics)

    for metric, row in profit_row_map.items():
        if metric in profit_results:
            values = profit_results[metric]
            for i, v in enumerate(values):
                cell = ws_profit.cell(row=row, column=2 + i)
                if pd.notna(v):
                    cell.value = float(v)
                    if metric in percentage_metrics:
                        cell.number_format = '0.0%'
                    else:
                        cell.number_format = '0.00'
                else:
                    cell.value = None

    '''
    for metric, row in row_map.items():
        if metric not in computed_results or computed_results[metric] is None:
            continue
        values = computed_results[metric]

        # 支持标量和 Series
        if isinstance(values, (int, float)):
            ws_formula.cell(row=row, column=2).value = values
        elif isinstance(values, pd.Series):
            for i, v in enumerate(values):
                ws_formula.cell(row=row, column=2 + i).value = v
        else:
            print(f"未知格式: {metric} -> {type(values)}")
    '''
    wb_formula.save(output_path)
    print(f"成功保存为: {output_path}")


# ==== 主程序 ====
if not inp.exists():
    raise FileNotFoundError(f"找不到文件: {inp}")

# 断开外链 -> 处理 -> 清理中间文件
temp_cleaned = break_external_links(str(inp))
try:
    process_excel(temp_cleaned, str(out))
finally:
    os.remove(temp_cleaned)
    print("已删除中间文件。")


成功保存为: E:\code\Simple-Bot-for-Stock-and-Financial-Report-Data-Processing\data\原始数据模板_财务分析_2025Q1_v1.xlsx
已删除中间文件。


In [ ]:
import pandas as pd

# 读取 Excel 文件
df = pd.read_excel(r'.\data\原始数据模板_财务分析_2025Q1.xlsx', header=None)
df = df.fillna(0)  # 如果你确认业务逻辑允许

# 检查第 2 列中感兴趣的行
rows_to_check = [37, 79, 105]
col_index = 1  # 第二列（索引从 0 开始）

for row in rows_to_check:
    value = df.iloc[row, col_index]
    print(f"第 {row+1} 行，第 2 列的值：{value}，类型：{type(value)}")
try:
    v1 = df.iloc[37]
    v2 = df.iloc[105]
    v3 = df.iloc[79]
    print(f"类型检查：d.iloc[37]={v1} ({type(v1)}), d.iloc[105]={v2} ({type(v2)}), d.iloc[79]={v3} ({type(v3)})")
    result = v1 / (v2 - v3)
    print(f"计算结果：{result}")
except Exception as e:
    print(f"计算 流动比率（扣除预收账款） 出错: {e}")


第 38 行，第 2 列的值：2325701337.46，类型：<class 'float'>
第 80 行，第 2 列的值：0，类型：<class 'int'>
第 106 行，第 2 列的值：1436504702.25，类型：<class 'float'>
类型检查：d.iloc[37]=0             流动资产合计
1      2325701337.46
2      2381879275.58
3      2211056219.37
4      2102845221.71
5       1998458114.9
6      2106340791.52
7      2011704741.89
8      2361985140.53
9      1652413558.91
10      1717018998.7
11      1618857233.9
12     1553672497.63
13     1478642357.82
14     1556904534.48
15     1681374992.68
16     1640783862.88
17     1660965477.53
18     1782056307.86
19     1430773452.11
20     1427531047.02
21     1431478980.08
22     1286892411.07
23     1220736237.81
24     1272326680.17
25     1267958806.99
26     1303136236.63
27     1443810903.03
28      1750658795.6
29     1580027963.43
30      779345291.09
31       718153932.8
32      623017726.04
33      676618099.72
34      644010032.59
35      579355505.95
36      739893405.57
37      714292708.61
38      740633752.25
39     1046227601.74
40     100807

In [32]:
import pandas as pd
import numpy as np

# 读取 Excel 文件
df = pd.read_excel(r'.\data\原始数据模板_财务分析_2025Q1.xlsx', header=None)
df = df.fillna(0)

# 定义同比计算函数（向后对比）
def yoy_series(row: pd.Series, offset=4):
    result = []
    for i in range(len(row)):
        if i + offset < len(row):
            denom = row[i + offset]
            if denom == 0 or pd.isna(denom):
                result.append(np.nan)
            else:
                result.append(row[i] / denom - 1)
        else:
            result.append(np.nan)
    return pd.Series(result)

# === 提取所需行数据（注意：从第二列开始，不含第一列）
row = df.iloc[14, 1:].reset_index(drop=True)


# 计算同比
yoy_result = yoy_series(row)

# 打印结果
print(yoy_result)


0    -0.032514
1     0.297011
2    -0.053473
3     0.122170
4     0.681700
5     0.372669
6     0.816707
7     0.800389
8     0.376523
9     0.661636
10   -0.008148
11   -0.076124
12   -0.192135
13   -0.103644
14   -0.014814
15   -0.053249
16   -0.098147
17   -0.188716
18    0.020113
19    0.062078
20    0.110853
21    0.001425
22   -0.036385
23    0.065230
24    0.149494
25    0.919489
26    0.720514
27    0.787959
28    0.334834
29   -0.019433
30    0.421638
31    0.551267
32    0.587834
33    0.944462
34    0.137444
35   -0.063262
36    0.126199
37   -0.046612
38    0.197084
39    0.626146
40    0.945176
41    2.235238
42    0.580251
43    0.634290
44    0.382042
45    0.834794
46    0.740094
47    0.484738
48    1.045165
49    1.196412
50    2.526946
51    2.388347
52    2.237318
53   -0.089079
54         NaN
55         NaN
56         NaN
57         NaN
dtype: float64


In [6]:
import pandas as pd
import numpy as np

# 读取 Excel 文件
df = pd.read_excel(r'.\data\原始数据模板_财务分析_2025Q1.xlsx', header=None)
df = df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce').fillna(0)
sheet_names = pd.ExcelFile(r'.\data\原始数据模板_财务分析_2025Q1.xlsx').sheet_names
second_sheet_name = sheet_names[1]  # 索引从0开始，1表示第二个sheet
df_profit_raw = pd.read_excel(r'.\data\原始数据模板_财务分析_2025Q1.xlsx', sheet_name=second_sheet_name, header=None)
print("df_profit.shape[0] =", df_profit_raw.shape[0])
df_profit = df_profit_raw.iloc[:, 1:].apply(pd.to_numeric, errors='coerce').fillna(0)
# 计算 应收变化/收入变化
def calc_receivable_change_ratio_multicol(df_main, df_profit):
    result = []
    for col in range(df_main.shape[1] - 1):  # 要防止 col+1 越界
        # === 分子：当前列应收 - 下一列应收 ===
        curr_sum = df_main.iloc[13:16, col].sum()
        next_sum = df_main.iloc[13:16, col + 1].sum()
        delta = curr_sum - next_sum  # ✅ 当前列减下一列

        # === 分母判断值
        current_flag = df_main.iloc[0, col]

        income_curr = df_profit.iloc[9, col]
        income_next = df_profit.iloc[9, col + 1]

        if current_flag == 1:
            base = income_curr
        else:
            base = income_curr - income_next  # ✅ 当前列减下一列

        if base == 0 or pd.isna(base):
            result.append(np.nan)
        else:
            result.append(delta / base)

    result.append(np.nan)  # 最后一列无法比较下一列，对应为空
    return pd.Series(result)




calc_receivable_change_ratio_multicol(df, df_profit)
    

df_profit.shape[0] = 79


0    -0.102923
1     0.088068
2    -0.078678
3     0.058831
4     0.053437
5    -0.061152
6    -0.000356
7     0.165272
8    -0.054857
9     0.059230
10   -0.001076
11    0.103426
12    0.031871
13   -0.162999
14   -0.013099
15    0.063985
16    0.070892
17   -0.152570
18   -0.062482
19    0.058016
20   -0.027821
21   -0.028930
22   -0.012479
23    0.038588
24   -0.050606
25   -0.009038
26    0.071292
27    0.123659
28    0.370962
29   -0.049497
30    0.030121
31   -0.017388
32    0.080224
33    0.163235
34    0.105640
35   -0.177083
36    0.130991
37   -0.107127
38    0.020478
39    0.059308
40    0.139394
41    0.046805
42    0.159239
43    0.130707
44   -0.073563
45   -0.290177
46    0.120461
47    0.041255
48   -0.120695
49   -0.238230
50    0.053928
51    0.140736
52   -0.105634
53   -0.072963
54    0.029877
55    0.062709
56    0.008927
57         NaN
dtype: float64